# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. Имеющиеся данные: технические характеристики, комплектации и цены автомобилей. 

Задача: построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

<div class="alert alert-info">Импортируем библиотеки и посмотрим, что у нас есть в данных
</div>

In [ ]:
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
import time
from sklearn.preprocessing import OrdinalEncoder
import warnings
warnings.simplefilter("ignore", UserWarning)

In [ ]:
df = pd.read_csv('/datasets/autos.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

<div class="alert alert-info">много пропусков в категориальных данных, тк мы не знаем, что там должно быть, заполним их неизвестным
</div>

In [ ]:
df = df.fillna('unknown')

<div class="alert alert-info">посмотрим, есть ли какая-нибудь корреляция
</div>

In [ ]:
df.corr()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
Price,1.000000,0.026916,0.158872,-0.333199,0.110581,NaN,0.076055
RegistrationYear,0.026916,1.000000,-0.000828,-0.053447,-0.011619,NaN,-0.003459
Power,0.158872,-0.000828,1.000000,0.024002,0.043380,NaN,0.021665
Kilometer,-0.333199,-0.053447,0.024002,1.000000,0.009571,NaN,-0.007698
RegistrationMonth,0.110581,-0.011619,0.043380,0.009571,1.000000,NaN,0.013995
NumberOfPictures,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PostalCode,0.076055,-0.003459,0.021665,-0.007698,0.013995,NaN,1.000000


<div class="alert alert-info">небольшая прямая корреляция цены и мощности, и непрямая корреляция побольше цены и километража, количество фоток никак не влияет; думаю, можно избавиться от следующих столбцов:
</div>

In [ ]:
df = df.drop(['RegistrationMonth', 'DateCreated', 'LastSeen', 'DateCrawled', 'NumberOfPictures', 'PostalCode'], axis=1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             354369 non-null  int64 
 1   VehicleType       354369 non-null  object
 2   RegistrationYear  354369 non-null  int64 
 3   Gearbox           354369 non-null  object
 4   Power             354369 non-null  int64 
 5   Model             354369 non-null  object
 6   Kilometer         354369 non-null  int64 
 7   FuelType          354369 non-null  object
 8   Brand             354369 non-null  object
 9   NotRepaired       354369 non-null  object
dtypes: int64(4), object(6)
memory usage: 27.0+ MB


In [ ]:
df['RegistrationYear'].unique()

array([1993, 2011, 2004, 2001, 2008, 1995, 1980, 2014, 1998, 2005, 1910,
       2016, 2007, 2009, 2002, 2018, 1997, 1990, 2017, 1981, 2003, 1994,
       1991, 1984, 2006, 1999, 2012, 2010, 2000, 1992, 2013, 1996, 1985,
       1989, 2015, 1982, 1976, 1983, 1973, 1111, 1969, 1971, 1987, 1986,
       1988, 1970, 1965, 1945, 1925, 1974, 1979, 1955, 1978, 1972, 1968,
       1977, 1961, 1960, 1966, 1975, 1963, 1964, 5000, 1954, 1958, 1967,
       1959, 9999, 1956, 3200, 1000, 1941, 8888, 1500, 2200, 4100, 1962,
       1929, 1957, 1940, 3000, 2066, 1949, 2019, 1937, 1951, 1800, 1953,
       1234, 8000, 5300, 9000, 2900, 6000, 5900, 5911, 1933, 1400, 1950,
       4000, 1948, 1952, 1200, 8500, 1932, 1255, 3700, 3800, 4800, 1942,
       7000, 1935, 1936, 6500, 1923, 2290, 2500, 1930, 1001, 9450, 1944,
       1943, 1934, 1938, 1688, 2800, 1253, 1928, 1919, 5555, 5600, 1600,
       2222, 1039, 9996, 1300, 8455, 1931, 1915, 4500, 1920, 1602, 7800,
       9229, 1947, 1927, 7100, 8200, 1946, 7500, 35

<div class="alert alert-info">так как у нас присутствуют очень странные значения, то, думаю, можно ограничиться годами с 1923 по 2022
</div>

In [ ]:
df = df[(df.RegistrationYear<2022)&(df.RegistrationYear>1923)]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354092 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             354092 non-null  int64 
 1   VehicleType       354092 non-null  object
 2   RegistrationYear  354092 non-null  int64 
 3   Gearbox           354092 non-null  object
 4   Power             354092 non-null  int64 
 5   Model             354092 non-null  object
 6   Kilometer         354092 non-null  int64 
 7   FuelType          354092 non-null  object
 8   Brand             354092 non-null  object
 9   NotRepaired       354092 non-null  object
dtypes: int64(4), object(6)
memory usage: 29.7+ MB


In [ ]:
df.Power.unique()

array([    0,   190,   163,    75,    69,   102,   109,    50,   125,
         101,   105,   140,   115,   131,    60,   136,   160,   231,
          90,   118,   193,    99,   113,   218,   122,   129,    70,
         306,    95,    61,   177,    80,   170,    55,   143,    64,
         286,   232,   150,   156,    82,   155,    54,   185,    87,
         180,    86,    84,   224,   235,   200,   178,   265,    77,
         110,   144,   120,   116,   184,   126,   204,    88,   194,
         305,   197,   179,   250,    45,   313,    41,   165,    98,
         130,   114,   211,    56,   201,   213,    58,   107,    83,
         174,   100,   220,    85,    73,   192,    68,    66,   299,
          74,    52,   147,    65,   310,    71,    97,   239,   203,
           5,   300,   103,   258,   320,    63,    81,   148,    44,
         145,   230,   280,   260,   104,   188,   333,   186,   117,
         141,    59,   132,   234,   158,    39,    92,    51,   135,
          53,   209,

In [ ]:
df.loc[(df.Power < 20) | (df.Power > 2000), 'Power'] = df.groupby('Model')['Power'].transform('median')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354092 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Price             354092 non-null  int64  
 1   VehicleType       354092 non-null  object 
 2   RegistrationYear  354092 non-null  int64  
 3   Gearbox           354092 non-null  object 
 4   Power             354092 non-null  float64
 5   Model             354092 non-null  object 
 6   Kilometer         354092 non-null  int64  
 7   FuelType          354092 non-null  object 
 8   Brand             354092 non-null  object 
 9   NotRepaired       354092 non-null  object 
dtypes: float64(1), int64(3), object(6)
memory usage: 37.8+ MB


<div class="alert alert-info"> оставим все цены больше 100
</div>

In [ ]:
df = df[(df.Price>100)]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 339877 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Price             339877 non-null  int64  
 1   VehicleType       339877 non-null  object 
 2   RegistrationYear  339877 non-null  int64  
 3   Gearbox           339877 non-null  object 
 4   Power             339877 non-null  float64
 5   Model             339877 non-null  object 
 6   Kilometer         339877 non-null  int64  
 7   FuelType          339877 non-null  object 
 8   Brand             339877 non-null  object 
 9   NotRepaired       339877 non-null  object 
dtypes: float64(1), int64(3), object(6)
memory usage: 28.5+ MB


## Обучение моделей

<div class="alert alert-info">обучим несколько моделей регресии
</div>

In [ ]:
df_2 = df.copy()
encoder = OrdinalEncoder()
df_2[["VehicleType", "Gearbox", "FuelType", "Model", "Brand", "NotRepaired"]] = encoder.fit_transform(df_2[["VehicleType", "Gearbox", "FuelType", "Model", "Brand", "NotRepaired"]])

In [ ]:
df_2.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
0,480,7.0,1993,1.0,95.0,116.0,150000,6.0,38.0,1.0
1,18300,2.0,2011,1.0,190.0,228.0,125000,2.0,1.0,2.0
2,9800,6.0,2004,0.0,163.0,117.0,125000,2.0,14.0,1.0
3,1500,5.0,2001,1.0,75.0,116.0,150000,6.0,38.0,0.0
4,3600,5.0,2008,1.0,69.0,101.0,90000,2.0,31.0,0.0


In [ ]:
df.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
0,480,unknown,1993,manual,95.0,golf,150000,petrol,volkswagen,unknown
1,18300,coupe,2011,manual,190.0,unknown,125000,gasoline,audi,yes
2,9800,suv,2004,auto,163.0,grand,125000,gasoline,jeep,unknown
3,1500,small,2001,manual,75.0,golf,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69.0,fabia,90000,gasoline,skoda,no


In [ ]:
df_1 = pd.get_dummies(df, drop_first=True)
pd.options.mode.chained_assignment = None


In [ ]:
df_1, df1_test = train_test_split(df_1, test_size=0.2, train_size=0.8)
df1_train, df1_valid = train_test_split(df_1, test_size=0.25, train_size=0.75)

In [ ]:
features1_test = df1_test.drop(['Price'], axis=1)
target1_test = df1_test['Price']
features1_train = df1_train.drop(['Price'], axis=1)
target1_train = df1_train['Price']
features1_valid = df1_valid.drop(['Price'], axis=1)
target1_valid = df1_valid['Price']

In [ ]:
numeric = ['RegistrationYear', 'Power', 'Kilometer']

scaler = StandardScaler()
scaler.fit(features1_train[numeric]) 
features1_train[numeric] = scaler.transform(features1_train[numeric])
features1_valid[numeric] = scaler.transform(features1_valid[numeric])
features1_test[numeric] = scaler.transform(features1_test[numeric])

In [ ]:
df_2, df2_test = train_test_split(df_2, test_size=0.2, train_size=0.8)
df2_train, df2_valid = train_test_split(df_2, test_size=0.25, train_size=0.75)

In [ ]:
features2_test = df2_test.drop(['Price'], axis=1)
target2_test = df2_test['Price']
features2_train = df2_train.drop(['Price'], axis=1)
target2_train = df2_train['Price']
features2_valid = df2_valid.drop(['Price'], axis=1)
target2_valid = df2_valid['Price']

In [ ]:
numeric = ['RegistrationYear', 'Power', 'Kilometer']

scaler = StandardScaler()
scaler.fit(features2_train[numeric]) 
features2_train[numeric] = scaler.transform(features2_train[numeric])
features2_valid[numeric] = scaler.transform(features2_valid[numeric])
features2_test[numeric] = scaler.transform(features2_test[numeric])

In [ ]:
#%%time
#start_time= time.time()
model = DecisionTreeRegressor(random_state=12345, max_depth=10) 
model.fit(features2_train, target2_train)
predictions1_valid = model.predict(features2_valid) 
print('RMSE DecisionTreeRegressor', mean_squared_error(target2_valid, predictions1_valid)**0.5)
#end_time= time.time()
#time_RMSE_dtr_train = end_time - start_time

RMSE DecisionTreeRegressor 2041.8110004507569


In [ ]:
#%%time
#start_time= time.time()
model = RandomForestRegressor(random_state=12345, n_estimators=10, max_depth=5) 
model.fit(features2_train, target2_train)
predictions2_valid = model.predict(features2_valid) 
print('RMSE RandomForestRegressor', mean_squared_error(target2_valid, predictions2_valid)**0.5)
#end_time= time.time()
#time_RMSE_rfr_train = end_time - start_time

RMSE RandomForestRegressor 2413.9242167515636


In [ ]:
#%%time
start_time= time.time()
model = LinearRegression() 
model.fit(features1_train, target1_train) 
predictions3_valid = model.predict(features1_valid) 

print('RMSE LinearRegression', mean_squared_error(target1_valid, predictions3_valid)**0.5)
#end_time= time.time()
#time_RMSE_lr_train = end_time - start_time

RMSE LinearRegression 2765.351841156755


In [ ]:
df, df_test = train_test_split(df, test_size=0.2, train_size=0.8)
df_train, df_valid = train_test_split(df, test_size=0.25, train_size=0.75)

In [ ]:
features_test = df_test.drop(['Price'], axis=1)
target_test = df_test['Price']
features_train = df_train.drop(['Price'], axis=1)
target_train = df_train['Price']
features_valid = df_valid.drop(['Price'], axis=1)
target_valid = df_valid['Price']

cat_features = ['VehicleType', 'Gearbox', 'Model',
                'FuelType', 'Brand', 'NotRepaired']

In [ ]:
df.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
199556,4800,convertible,1999,auto,193.0,unknown,150000,lpg,toyota,unknown
322964,15999,unknown,2017,unknown,75.0,unknown,125000,unknown,bmw,unknown
113795,13900,wagon,2012,manual,80.0,golf,50000,petrol,volkswagen,no
333944,450,small,1998,manual,60.0,corsa,150000,petrol,opel,unknown
330236,19999,wagon,2010,auto,239.0,a4,150000,gasoline,audi,no


In [ ]:
#%%time
#start_time= time.time()
model = CatBoostRegressor(loss_function="RMSE", iterations=100, learning_rate=1, depth=6)

model.fit(features_train, target_train, cat_features=cat_features, verbose=10)

probabilities_valid = model.predict(features_valid)

print(mean_squared_error(target_valid, probabilities_valid)**0.5)
#end_time= time.time()
#time_RMSE_cbr_train = end_time - start_time

0:	learn: 2641.3121466	total: 155ms	remaining: 15.3s
10:	learn: 1939.6681342	total: 925ms	remaining: 7.48s
20:	learn: 1833.0908370	total: 1.64s	remaining: 6.19s
30:	learn: 1782.3563399	total: 2.37s	remaining: 5.27s
40:	learn: 1747.7946597	total: 3.09s	remaining: 4.44s
50:	learn: 1723.8487132	total: 3.8s	remaining: 3.65s
60:	learn: 1702.7945756	total: 4.52s	remaining: 2.89s
70:	learn: 1689.0559530	total: 5.22s	remaining: 2.13s
80:	learn: 1673.0363431	total: 5.92s	remaining: 1.39s
90:	learn: 1663.2385631	total: 6.63s	remaining: 655ms
99:	learn: 1655.2194851	total: 7.27s	remaining: 0us
1732.5660818240046


In [ ]:
#%%time
#start_time= time.time()
model=LGBMRegressor(learning_rate=1, max_depth=4)

model.fit(features2_train, target2_train, categorical_feature=cat_features)

valid_preds = model.predict(features2_valid)
print(mean_squared_error(target2_valid, valid_preds)**0.5)
#end_time= time.time()
#time_RMSE_lgbm_train = end_time - start_time

1771.0099604285692


## Анализ моделей

In [ ]:
%%time
start_time= time.time()
model = DecisionTreeRegressor(random_state=12345, max_depth=10) 
model.fit(features2_train, target2_train)
end_time_train = time.time()
time_RMSE_dtr_train = end_time_train - start_time
predicted_test_dtr = model.predict(features2_test)
result_dtr = mean_squared_error(target2_test, predicted_test_dtr)**0.5
print(result_dtr)
end_time= time.time()
time_RMSE_dtr= end_time - end_time_train

2038.6059489773854
CPU times: user 391 ms, sys: 890 µs, total: 392 ms
Wall time: 407 ms


In [ ]:
%%time
start_time= time.time()
model = RandomForestRegressor(random_state=12345, n_estimators=50, max_depth=10) 
model.fit(features2_train, target2_train)
end_time_train = time.time()
time_RMSE_rfr_train = end_time_train - start_time
predicted_test_rfr = model.predict(features2_test) 
result_rfr = mean_squared_error(target2_test, predicted_test_rfr)**0.5
print(result_rfr)
end_time= time.time()
time_RMSE_rfr= end_time - end_time_train

1920.0198978158483
CPU times: user 12.9 s, sys: 0 ns, total: 12.9 s
Wall time: 12.9 s


In [ ]:
%%time
start_time= time.time()
model = LinearRegression() 
model.fit(features1_train, target1_train)
end_time_train = time.time()
time_RMSE_lr_train = end_time_train - start_time
predicted_test_lr = model.predict(features1_test) 
result_lr = mean_squared_error(target1_test, predicted_test_lr)**0.5
print(result_lr)
end_time= time.time()
time_RMSE_lr= end_time - end_time_train

2732.3529157361777
CPU times: user 19.3 s, sys: 10 s, total: 29.3 s
Wall time: 29.3 s


In [ ]:
%%time
start_time= time.time()
model = CatBoostRegressor(iterations=100, learning_rate=1) 
model.fit(features_train, target_train, cat_features=cat_features, verbose=10)
end_time_train = time.time()
time_RMSE_cbr_train = end_time_train - start_time
predicted_test_cbr = model.predict(features_test) 
result_cbr = mean_squared_error(target_test, predicted_test_cbr)**0.5
print(result_cbr)
end_time= time.time()
time_RMSE_cbr= end_time - end_time_train

0:	learn: 2641.3121466	total: 103ms	remaining: 10.2s
10:	learn: 1939.6681342	total: 923ms	remaining: 7.47s
20:	learn: 1833.0908370	total: 1.69s	remaining: 6.35s
30:	learn: 1782.3563399	total: 2.47s	remaining: 5.49s
40:	learn: 1747.7946597	total: 3.25s	remaining: 4.68s
50:	learn: 1723.8487132	total: 4.04s	remaining: 3.88s
60:	learn: 1702.7945756	total: 4.82s	remaining: 3.08s
70:	learn: 1689.0559530	total: 5.57s	remaining: 2.27s
80:	learn: 1673.0363431	total: 6.35s	remaining: 1.49s
90:	learn: 1663.2385631	total: 7.09s	remaining: 701ms
99:	learn: 1655.2194851	total: 7.77s	remaining: 0us
1726.9916419217534
CPU times: user 8.2 s, sys: 35.3 ms, total: 8.24 s
Wall time: 9.43 s


In [ ]:
%%time
start_time= time.time()
model = LGBMRegressor(learning_rate=1, max_depth=4)
model.fit(features2_train, target2_train, categorical_feature=cat_features)
end_time_train = time.time()
time_RMSE_lgbm_train = end_time_train - start_time
predicted_test_lgbm = model.predict(features2_test) 
result_lgbm = mean_squared_error(target2_test, predicted_test_lgbm)**0.5
print(result_lgbm)
end_time= time.time()
time_RMSE_lgbm= end_time - end_time_train

1779.8055977559645
CPU times: user 14.1 s, sys: 80.4 ms, total: 14.2 s
Wall time: 14.2 s


In [ ]:
df_voila = pd.DataFrame({'модель': ['DecisionTreeRegressor', 'RandomForestRegressor', 'LinearRegression', 'CatBoostRegressor', 'LGBM Regressor'],
                         'время обучения (с)': [time_RMSE_dtr_train, time_RMSE_rfr_train, time_RMSE_lr_train, time_RMSE_cbr_train, time_RMSE_lgbm_train],
                         'время предсказания (c)': [time_RMSE_dtr, time_RMSE_rfr, time_RMSE_lr, time_RMSE_cbr, time_RMSE_lgbm],
                         'RMSE': [result_dtr, result_rfr, result_lr, result_cbr, result_lgbm]})

In [ ]:
display(df_voila)

,модель,время обучения (с),время предсказания (c),RMSE
0,DecisionTreeRegressor,0.393423,0.013473,2038.605949
1,RandomForestRegressor,12.651493,0.289699,1920.019898
2,LinearRegression,29.134376,0.130051,2732.352916
3,CatBoostRegressor,9.349045,0.083059,1726.991642
4,LGBM Regressor,13.684433,0.499548,1779.805598


<div class="alert alert-info">Итого:
    
1) был проведен анализ данных, удалены странные выбивающиеся значения

2) обучено и проверено 5 моделей: Decision Tree Regressor, Random FOrest Regressor, Linear Regression, CatBoost Regressor, LGBM Regressor
    
3) посчитано время выполнения предсказания и RMSE
</div>